# TLA+ Demo: LLM-Assisted Specification

**TLA+** is a formal specification language for concurrent and distributed systems. Unlike Dafny and Lean, TLA+ uses **model checking**: it exhaustively explores all possible states to find bugs.

In this notebook, we'll:
1. See what a TLA+ specification looks like
2. Ask an LLM to generate a spec from natural language
3. Run TLC (the model checker) to find counterexamples
4. Feed counterexamples back to the LLM to fix the spec

## Why TLA+?

TLA+ is used at Amazon, Microsoft, and other companies for distributed systems:
- AWS uses it to verify DynamoDB, S3, and other services
- It catches subtle concurrency bugs that testing misses
- The "genefication" workflow (from [Murat Demirbas's blog](https://www.mydistributed.systems/2025/01/genefication.html)) shows how LLMs can draft TLA+ specs

## The Genefication Workflow

```
English description
       ↓
LLM generates TLA+ spec
       ↓
TLC model checker runs
       ↓
   ┌───┴───┐
   ↓       ↓
  Pass   Counterexample found
   ↓           ↓
 Done    Feed counterexample to LLM
               ↓
         LLM fixes spec
               ↓
         (repeat)
```

In [ ]:
import sys
sys.path.insert(0, '..')

from src.llm_client import LLMClient
from src.verifiers import TLCVerifier
from rich import print as rprint
from rich.syntax import Syntax
from rich.panel import Panel

client = LLMClient()

try:
    verifier = TLCVerifier()
    print("✅ Client and TLC verifier ready")
except RuntimeError as e:
    print(f"⚠️ TLA+ tools not installed: {e}")
    print("Run ./setup.sh to install TLA+ tools")
    verifier = None

## Example 1: Mutual Exclusion

Let's start with the classic: Peterson's algorithm for mutual exclusion.

**Problem**: Two processes want to access a critical section. We need to ensure:
1. **Safety**: They never both enter at the same time
2. **Liveness**: If a process wants in, it eventually gets in

In [ ]:
# Load the reference implementation
with open('../examples/tlaplus/MutualExclusion.tla') as f:
    mutex_spec = f.read()

rprint(Panel(Syntax(mutex_spec, 'text', theme='monokai'), title="Peterson's Algorithm in TLA+"))

In [ ]:
# Verify with TLC
if verifier:
    with open('../examples/tlaplus/MutualExclusion.cfg') as f:
        config = f.read()
    
    result = verifier.verify(mutex_spec, config)
    print(f"Verification: {'✅ PASSED' if result.success else '❌ FAILED'}")
    if not result.success:
        print(f"Counterexample:\n{result.error}")

## Generating Specs from Natural Language

Now let's ask the LLM to generate a TLA+ spec from scratch, given only an English description.

In [ ]:
description = """
Design a simple counter system with the following properties:

1. There is a counter variable that starts at 0
2. Two processes can increment the counter
3. The counter should never exceed 5
4. Each process can only increment when it's their turn
5. Processes take turns (alternating)

Verify:
- Safety: counter never exceeds 5
- The system can make progress (liveness)
"""

print("Description:")
print(description)

In [ ]:
def genefication_loop(description: str, max_attempts: int = 5):
    """Run the genefication workflow: LLM generates, TLC checks, iterate."""
    
    for attempt in range(1, max_attempts + 1):
        print(f"\n{'='*60}")
        print(f"Attempt {attempt}/{max_attempts}")
        print('='*60)
        
        # Ask LLM to generate/fix spec
        if attempt == 1:
            print("\n📤 Asking LLM to generate TLA+ spec...")
            response = client.generate_tlaplus_spec(description)
        else:
            print(f"\n📤 Asking LLM to fix based on counterexample...")
            response = client.generate_tlaplus_spec(description, error=result.error)
        
        spec = client._extract_code(response.content)
        print(f"   Tokens used: {response.input_tokens + response.output_tokens}")
        
        rprint(Panel(Syntax(spec, 'text', theme='monokai', line_numbers=True), 
                     title=f"Generated Spec (Attempt {attempt})"))
        
        if verifier is None:
            print("⚠️ TLC not available, showing LLM output only")
            return spec, attempt
        
        # Verify with TLC
        print("\n🔍 Running TLC model checker...")
        result = verifier.verify(spec)
        
        if result.success:
            print("\n✅ MODEL CHECKING PASSED!")
            return spec, attempt
        else:
            print(f"\n❌ Model checking found issue:")
            print(result.error[:1000])  # Truncate long counterexamples
    
    print(f"\n⚠️ Max attempts ({max_attempts}) reached")
    return spec, max_attempts

In [ ]:
# Run the genefication loop
final_spec, attempts = genefication_loop(description)

## How TLA+ Differs

| Aspect | Dafny/Lean | TLA+ |
|--------|-----------|------|
| Verification | Theorem proving | Model checking |
| Feedback | Proof obligations | Counterexamples (traces) |
| Strength | Proofs are general | Finds actual bug scenarios |
| Weakness | May need complex invariants | State space explosion |
| LLM use | Generate proofs/tactics | Generate entire specs |

## Key Insight: Counterexamples are Gold

TLC's counterexamples are incredibly useful for LLMs:
- They show a **concrete execution trace** that violates the property
- The LLM can see exactly which state transition caused the problem
- This is more actionable than abstract proof obligations

## From the FM-ALPACA Paper

The [FM-ALPACA benchmark](https://arxiv.org/abs/2501.16207) includes TLA+ as one of five languages:

> "We constructed 18k high-quality instruction-response pairs across five formal specification languages (Coq, Lean4, Dafny, ACSL, and **TLA+**)."

TLA+ is particularly interesting because:
- It's used in industry more than academic proof assistants
- The model checking feedback loop is natural for LLMs
- PlusCal (an algorithmic notation that compiles to TLA+) is more intuitive

## Next: Comparison

Continue to [05-comparison.ipynb](05-comparison.ipynb) to see a side-by-side evaluation of all three tools.